# import 및 drive 연결

In [1]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# 데이터 불러오기

In [3]:
traveler = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/002.권역별데이터(MBTI예측)/여행객(MBTI예측).csv')
codeA = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/000.국내 여행로그 데이터(동부권)/01-1.정식개방데이터/Training/01.원천데이터/TS_csv/tc_codea_코드A.csv')
codeB = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/000.국내 여행로그 데이터(동부권)/01-1.정식개방데이터/Training/01.원천데이터/TS_csv/tc_codeb_코드B.csv')
sigungu = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/000.국내 여행로그 데이터(동부권)/01-1.정식개방데이터/Training/01.원천데이터/TS_csv/tc_sgg_시군구코드.csv')

# columns 삭제

In [4]:
traveler.drop(['EDU_NM', 'EDU_FNSH_SE', 'MARR_STTS', 'FAMILY_MEMB', 'JOB_NM', 'JOB_ETC',
               'INCOME', 'HOUSE_INCOME', 'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION',
               'TRAVEL_STATUS_YMD'], axis=1, inplace=True)
traveler

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,...,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,MBTI
0,a006685,41,여,30,3,6,41,41650,28,28720,...,1,1,5,2,2인 가족 여행,10.0,NaN,NaN,1,INFP
1,a014274,11,여,20,3,5,50,50110,47,47130,...,3,3,2,6,나홀로 여행,4.0,7.0,1.0,0,ESTJ
2,a001158,41,남,30,2,2,11,11680,48,48120,...,2,2,3,2,나홀로 여행,8.0,1.0,6.0,0,ENFJ
3,a005594,28,남,30,2,2,11,11500,28,28110,...,6,6,6,2,자녀 동반 여행,3.0,1.0,2.0,3,ENFP
4,a007033,11,남,20,1,1,41,41450,11,11710,...,3,1,7,7,나홀로 여행,1.0,2.0,7.0,0,ENFP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,d000922,41,남,50,3,2,42,42150,45,45180,...,2,3,3,3,부모 동반 여행,1.0,3.0,2.0,4,ISFJ
3800,d010626,50,여,40,3,2,11,11710,41,41460,...,6,6,5,2,자녀 동반 여행,1.0,2.0,3.0,3,ESFP
3801,d001143,11,여,30,2,3,50,50130,42,42210,...,3,5,5,5,자녀 동반 여행,1.0,7.0,3.0,3,ESFP
3802,b004853,48,여,30,2,2,26,26530,48,48220,...,2,3,2,6,나홀로 여행,2.0,1.0,5.0,0,ISTJ


# 데이터 라벨링 복구

주소지는 sigungu파일의 코드와 대조하여 변환\
카테고리는 codeb파일의 코드와 대조하여 변환(codea는 ERD를 보면 나와있음)

In [5]:
def merge_sido(sigungu, df, merge_on_col, rename_col):
    sigungu_rev = sigungu[['SGG_CD1', 'SIDO_NM']]
    sigungu_rev.drop_duplicates(inplace=True)
    df = pd.merge(sigungu_rev, df, left_on='SGG_CD1', right_on=merge_on_col, how='right')
    df.drop(columns=['SGG_CD1', merge_on_col], inplace=True)
    df.rename(columns={'SIDO_NM': rename_col}, inplace=True)
    return df

# 거주지 수정
merge_df = merge_sido(sigungu, traveler, 'RESIDENCE_SGG_CD', '거주지')

# 선호여행(시도) 수정
merge_df = merge_sido(sigungu, merge_df, 'TRAVEL_LIKE_SIDO_1', '선호여행1(시도)')
merge_df = merge_sido(sigungu, merge_df, 'TRAVEL_LIKE_SIDO_2', '선호여행2(시도)')
merge_df = merge_sido(sigungu, merge_df, 'TRAVEL_LIKE_SIDO_3', '선호여행3(시도)')

def merge_sigungu(sigungu, df, merge_on_col, rename_col):
    sigungu_rev = sigungu[['SGG_CD1', 'SGG_CD2', 'SGG_NM']]
    sigungu_rev = sigungu_rev[sigungu_rev['SGG_CD2'].notna()]
    sigungu_rev['SGG_CD2'] = sigungu_rev['SGG_CD1'].astype(str) + sigungu_rev['SGG_CD2'].astype(str)
    sigungu_rev['SGG_CD2'] = sigungu_rev['SGG_CD2'].astype(float)
    sigungu_rev.drop_duplicates(inplace=True)

    df = pd.merge(sigungu_rev, df, left_on='SGG_CD2', right_on=merge_on_col, how='right')
    df.drop(columns=['SGG_CD1', 'SGG_CD2'], inplace=True)
    df.rename(columns={'SGG_NM': rename_col, merge_on_col: f'{rename_col}코드'}, inplace=True)
    return df

# 선호여행(시군구) 수정, 결측치처리를 위해 시군구 코드 남김
merge_df = merge_sigungu(sigungu, merge_df, 'TRAVEL_LIKE_SGG_1', '선호여행1(시군구)')
merge_df = merge_sigungu(sigungu, merge_df, 'TRAVEL_LIKE_SGG_2', '선호여행2(시군구)')
merge_df = merge_sigungu(sigungu, merge_df, 'TRAVEL_LIKE_SGG_3', '선호여행3(시군구)')


def merge_codeB(df, code_a, col_changename, merge_on_col):
    codeB_rev = codeB.loc[codeB['cd_a'] == code_a][['cd_nm', 'cd_b']]
    codeB_rev['cd_b'] = codeB_rev['cd_b'].astype(int)
    df = pd.merge(codeB_rev, df, left_on='cd_b', right_on=merge_on_col, how='right')
    df.drop(columns=['cd_b', merge_on_col], inplace=True)
    df.rename(columns={'cd_nm': col_changename}, inplace=True)
    return df

# 여행빈도(기간) 수정
merge_df = merge_codeB(merge_df, 'TTM', '여행빈도(기간)', 'TRAVEL_TERM')

# 여행동기 수정
merge_df = merge_codeB(merge_df, 'TMT', '여행동기1', 'TRAVEL_MOTIVE_1')
merge_df = merge_codeB(merge_df, 'TMT', '여행동기2', 'TRAVEL_MOTIVE_2')
merge_df = merge_codeB(merge_df, 'TMT', '여행동기3', 'TRAVEL_MOTIVE_3')

def merge_travelstyle(df, style_col):
    df.loc[df[style_col]==1, style_col] = '전자 매우선호'
    df.loc[df[style_col]==2, style_col] = '전자 중간선호'
    df.loc[df[style_col]==3, style_col] = '전자 조금선호'
    df.loc[df[style_col]==5, style_col] = '후자 조금선호'
    df.loc[df[style_col]==6, style_col] = '후자 중간선호'
    df.loc[df[style_col]==7, style_col] = '후자 매우선호'
    return df

# 여행스타일 수정
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_1')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_2')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_3')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_4')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_5')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_6')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_7')
merge_df = merge_travelstyle(merge_df, 'TRAVEL_STYL_8')


# 여행객 컬럼명 수정
merge_df.rename(columns={'GENDER':'성별', 'AGE_GRP':'연령', 'TRAVEL_NUM':'여행빈도', 'TRAVEL_STATUS_ACCOMPANY':'여행현황(동반)', 'TRAVEL_COMPANIONS_NUM':'여행동반자수',
                         'TRAVEL_STYL_1':'자연/도시', 'TRAVEL_STYL_2':'숙박/당일', 'TRAVEL_STYL_3':'익숙한/새로운',
                         'TRAVEL_STYL_4':'편하지만비싼숙소/불편하지만저렴한숙소', 'TRAVEL_STYL_5':'휴양휴식/체험활동', 'TRAVEL_STYL_6':'잘알려지지않은/알려진',
                         'TRAVEL_STYL_7':'계획에따른/상황에따른','TRAVEL_STYL_8':'사진촬영중요하지않음/사진촬영중요'}, inplace=True)

merge_df

,여행동기3,여행동기2,여행동기1,여행빈도(기간),선호여행3(시군구),선호여행2(시군구),선호여행1(시군구),선호여행3(시도),선호여행2(시도),선호여행1(시도),...,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,여행현황(동반),여행동반자수,MBTI
0,NaN,NaN,기타,1년,영월군,옹진군,포천시,강원도,인천광역시,경기도,...,전자 매우선호,후자 매우선호,후자 조금선호,전자 매우선호,전자 매우선호,후자 조금선호,전자 중간선호,2인 가족 여행,1,INFP
1,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,1년,원주시,경주시,제주시,강원도,경상북도,제주특별자치도,...,전자 매우선호,전자 매우선호,후자 조금선호,전자 조금선호,전자 조금선호,전자 중간선호,후자 중간선호,나홀로 여행,0,ESTJ
2,"운동, 건강 증진 및 충전","일상적인 환경 및 역할에서의 탈출, 지루함 탈피","역사 탐방, 문화적 경험 등 교육적 동기",한달,제주시,NaN,강남구,제주특별자치도,경상남도,서울특별시,...,전자 조금선호,전자 중간선호,전자 조금선호,전자 중간선호,전자 중간선호,전자 조금선호,전자 중간선호,나홀로 여행,0,ENFJ
3,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,한달,김포시,중구,강서구,경기도,인천광역시,서울특별시,...,후자 매우선호,전자 중간선호,후자 중간선호,후자 중간선호,후자 중간선호,후자 중간선호,전자 중간선호,자녀 동반 여행,3,ENFP
4,새로운 경험 추구,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1주일,용산구,송파구,하남시,서울특별시,서울특별시,경기도,...,후자 매우선호,전자 매우선호,전자 매우선호,전자 조금선호,전자 매우선호,후자 매우선호,후자 매우선호,나홀로 여행,0,ENFP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행 동반자와의 친밀감 및 유대감 증진,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1년,서귀포시,정읍시,강릉시,제주특별자치도,전라북도,강원도,...,전자 매우선호,전자 매우선호,전자 조금선호,전자 중간선호,전자 조금선호,전자 조금선호,전자 조금선호,부모 동반 여행,4,ISFJ
3800,여행 동반자와의 친밀감 및 유대감 증진,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1년,해운대구,NaN,송파구,부산광역시,경기도,서울특별시,...,전자 중간선호,전자 매우선호,후자 중간선호,후자 중간선호,후자 중간선호,후자 조금선호,전자 중간선호,자녀 동반 여행,3,ESFP
3801,여행 동반자와의 친밀감 및 유대감 증진,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",한달,가평군,속초시,서귀포시,경기도,강원도,제주특별자치도,...,전자 중간선호,전자 조금선호,전자 조금선호,전자 조금선호,후자 조금선호,후자 조금선호,후자 조금선호,자녀 동반 여행,3,ESFP
3802,SNS 사진 등록 등 과시,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",한달,거제시,통영시,사상구,경상남도,경상남도,부산광역시,...,전자 중간선호,전자 조금선호,전자 조금선호,전자 중간선호,전자 조금선호,전자 중간선호,후자 중간선호,나홀로 여행,0,ISTJ


# 데이터 전처리

## 선호여행(시군구) 결측치 처리

확인결과 '시군구코드' + 1을 하면 '시'데이터를 얻을 수 있음\
예) 경상남도 NaN 48120\
경상남도 창원시 의창구 48121\
 \
시 + 구로 변환되기 때문에 띄워쓰기를 중심으로 split하여 0번째로 컬럼 기입

In [6]:
# 선호여행1(시군구) 결측치 확인
merge_df.loc[merge_df['선호여행1(시군구)'].isna()]['선호여행1(시군구)코드']

7       41110
12      41110
62      45110
72      41130
140     48120
        ...  
3528    41460
3634    48120
3706    47110
3710    41280
3723    41280
Name: 선호여행1(시군구)코드, Length: 149, dtype: int64

In [7]:
def process_missing_values(df, col_name, col_code):
    # 결측치 처리: 시군구코드 + 1하여 변환
    df.loc[df[col_name].isna(), col_code] += 1

    # 시군구코드 수정
    sigungu_rev2 = sigungu[['SGG_CD1', 'SGG_CD2', 'SGG_NM']]
    sigungu_rev2 = sigungu_rev2[sigungu_rev2['SGG_CD2'].notna()]
    sigungu_rev2['SGG_CD2'] = sigungu_rev2['SGG_CD1'].astype(str) + sigungu_rev2['SGG_CD2'].astype(str)
    sigungu_rev2['SGG_CD2'] = sigungu_rev2['SGG_CD2'].astype(float)
    sigungu_rev2.drop_duplicates(inplace=True)
    df = pd.merge(sigungu_rev2, df, left_on='SGG_CD2', right_on=col_code, how='right')

    # 시군구명 채우기
    df[col_name] = df[col_name].fillna(df['SGG_NM'])
    df[col_name] = df.apply(lambda row: row['SGG_NM'].split()[0] if pd.isna(row[col_name]) else row[col_name], axis=1)

    # 불필요한 열 제거
    df.drop(columns=['SGG_CD1', 'SGG_CD2', 'SGG_NM', col_code], inplace=True)

    return df

# 선호여행1(시군구) 처리
merge_df = process_missing_values(merge_df, '선호여행1(시군구)', '선호여행1(시군구)코드')
merge_df = process_missing_values(merge_df, '선호여행2(시군구)', '선호여행2(시군구)코드')
merge_df = process_missing_values(merge_df, '선호여행3(시군구)', '선호여행3(시군구)코드')

merge_df.loc[merge_df[['선호여행1(시군구)', '선호여행2(시군구)', '선호여행3(시군구)']].isna().any(axis=1)]

,여행동기3,여행동기2,여행동기1,여행빈도(기간),선호여행3(시군구),선호여행2(시군구),선호여행1(시군구),선호여행3(시도),선호여행2(시도),선호여행1(시도),...,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,여행현황(동반),여행동반자수,MBTI


## 여행지(시도), 여행지(시군구) 컬럼 합치기

부산광역시, 금정구 -> 부산광역시\
강원도, 속초시 -> 속초시

In [8]:
merge_df['선호여행1(시도)'].unique()

array(['경기도', '제주특별자치도', '서울특별시', '강원도', '충청남도', '인천광역시', '부산광역시', '경상북도',
       '대구광역시', '경상남도', '전라북도', '전라남도', '대전광역시', '충청북도', '광주광역시', '울산광역시'],
      dtype=object)

In [9]:
# '광역시', '특별시', '특별자치도'를 포함하지 않은 경우, 선호여행(시군구) 값으로 선호여행(시도) 업데이트
# 선호여행(시군구) 컬럼 삭제
for idx in range(1, 4):
  merge_df.loc[~merge_df[f'선호여행{idx}(시도)'].str.contains('광역시|특별시|특별자치도', na=False), f'선호여행{idx}(시도)'] = merge_df[f'선호여행{idx}(시군구)']
  merge_df[f'선호여행{idx}(시도)']=merge_df.apply(lambda row: row[f'선호여행{idx}(시도)'].split()[0], axis=1)

  merge_df.drop(columns=[f'선호여행{idx}(시군구)'], inplace=True)

In [10]:
merge_df.rename(columns={'선호여행1(시도)':'선호여행1', '선호여행2(시도)':'선호여행2', '선호여행3(시도)':'선호여행3'}, inplace=True)

## 컬럼 정리

In [11]:
merge_df.columns

Index(['여행동기3', '여행동기2', '여행동기1', '여행빈도(기간)', '선호여행3', '선호여행2', '선호여행1', '거주지',
       'TRAVELER_ID', '성별', '연령', '여행빈도', '자연/도시', '숙박/당일', '익숙한/새로운',
       '편하지만비싼숙소/불편하지만저렴한숙소', '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른',
       '사진촬영중요하지않음/사진촬영중요', '여행현황(동반)', '여행동반자수', 'MBTI'],
      dtype='object')

In [13]:
merge_df = merge_df[['TRAVELER_ID', '성별', '연령', '거주지', 'MBTI', '여행빈도', '여행빈도(기간)', '여행현황(동반)', '여행동반자수',
                     '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소', '휴양휴식/체험활동', '잘알려지지않은/알려진',
                     '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요', '여행동기1', '여행동기2', '여행동기3',
                     '선호여행1', '선호여행2', '선호여행3']]

In [14]:
merge_df

,TRAVELER_ID,성별,연령,거주지,MBTI,여행빈도,여행빈도(기간),여행현황(동반),여행동반자수,자연/도시,...,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,여행동기1,여행동기2,여행동기3,선호여행1,선호여행2,선호여행3
0,a006685,여,30,경기도,INFP,6,1년,2인 가족 여행,1,전자 매우선호,...,전자 매우선호,전자 매우선호,후자 조금선호,전자 중간선호,기타,NaN,NaN,포천시,인천광역시,영월군
1,a014274,여,20,서울특별시,ESTJ,5,1년,나홀로 여행,0,전자 중간선호,...,전자 조금선호,전자 조금선호,전자 중간선호,후자 중간선호,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",제주특별자치도,경주시,원주시
2,a001158,남,30,경기도,ENFJ,2,한달,나홀로 여행,0,전자 매우선호,...,전자 중간선호,전자 중간선호,전자 조금선호,전자 중간선호,"역사 탐방, 문화적 경험 등 교육적 동기","일상적인 환경 및 역할에서의 탈출, 지루함 탈피","운동, 건강 증진 및 충전",서울특별시,창원시,제주특별자치도
3,a005594,남,30,인천광역시,ENFP,2,한달,자녀 동반 여행,3,전자 중간선호,...,후자 중간선호,후자 중간선호,후자 중간선호,전자 중간선호,여행 동반자와의 친밀감 및 유대감 증진,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",서울특별시,인천광역시,김포시
4,a007033,남,20,서울특별시,ENFP,1,1주일,나홀로 여행,0,후자 매우선호,...,전자 조금선호,전자 매우선호,후자 매우선호,후자 매우선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,하남시,서울특별시,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,d000922,남,50,경기도,ISFJ,2,1년,부모 동반 여행,4,후자 조금선호,...,전자 중간선호,전자 조금선호,전자 조금선호,전자 조금선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",강릉시,정읍시,제주특별자치도
3800,d010626,여,40,제주특별자치도,ESFP,2,1년,자녀 동반 여행,3,후자 중간선호,...,후자 중간선호,후자 중간선호,후자 조금선호,전자 중간선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행 동반자와의 친밀감 및 유대감 증진,서울특별시,용인시,부산광역시
3801,d001143,여,30,서울특별시,ESFP,3,한달,자녀 동반 여행,3,전자 조금선호,...,전자 조금선호,후자 조금선호,후자 조금선호,후자 조금선호,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,제주특별자치도,속초시,가평군
3802,b004853,여,30,경상남도,ISTJ,2,한달,나홀로 여행,0,전자 중간선호,...,전자 중간선호,전자 조금선호,전자 중간선호,후자 중간선호,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",SNS 사진 등록 등 과시,부산광역시,통영시,거제시


# csv추출

In [15]:
merge_df.to_csv(f'/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/004.전처리된데이터/여행객.csv', index=False)